# DECISION TREE

İkinci proje olan regresyon projesinde ve üçüncü proje olan sınıflandırma projesinde kullandığınız veri kümelerini tekrar kullanarak, karar ağaçları ile regresyon ve sınıflandırma modelleri oluşturun ve bu modeller için en uygun parametre değerlerini belirleyin. En ideal parametreyi bulmak için daha önceki bölümde öğrendiğiniz çapraz doğrulama metodlarını kullanabilirsiniz.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats.mstats import winsorize
import scipy.stats as stats
from sklearn import linear_model
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error,explained_variance_score
from sklearn.metrics import mean_squared_error as mse
from statsmodels.tools.eval_measures import mse, rmse
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report, precision_recall_fscore_support
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import log_loss
from sklearn.utils import resample
from sklearn.model_selection import KFold, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVC
from sklearn.svm import SVR
import xgboost as xgb
import lightgbm as lgb

In [2]:
life_df=pd.read_csv("Life Expectancy Data.csv")
life_df.head()

,Country,Year,Status,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,...,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
0,Afghanistan,2015,Developing,65.0,263.0,62,0.01,71.279624,65.0,1154,...,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1
1,Afghanistan,2014,Developing,59.9,271.0,64,0.01,73.523582,62.0,492,...,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0
2,Afghanistan,2013,Developing,59.9,268.0,66,0.01,73.219243,64.0,430,...,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9
3,Afghanistan,2012,Developing,59.5,272.0,69,0.01,78.184215,67.0,2787,...,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8
4,Afghanistan,2011,Developing,59.2,275.0,71,0.01,7.097109,68.0,3013,...,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5


In [3]:
life_df2=life_df.copy()

In [4]:
life_df2.columns.values

array(['Country', 'Year', 'Status', 'Life expectancy ', 'Adult Mortality',
       'infant deaths', 'Alcohol', 'percentage expenditure',
       'Hepatitis B', 'Measles ', ' BMI ', 'under-five deaths ', 'Polio',
       'Total expenditure', 'Diphtheria ', ' HIV/AIDS', 'GDP',
       'Population', ' thinness  1-19 years', ' thinness 5-9 years',
       'Income composition of resources', 'Schooling'], dtype=object)

Words have space

In [5]:
life_df2.columns = life_df2.columns.str.strip()

In [6]:
life_df2.columns.values

array(['Country', 'Year', 'Status', 'Life expectancy', 'Adult Mortality',
       'infant deaths', 'Alcohol', 'percentage expenditure',
       'Hepatitis B', 'Measles', 'BMI', 'under-five deaths', 'Polio',
       'Total expenditure', 'Diphtheria', 'HIV/AIDS', 'GDP', 'Population',
       'thinness  1-19 years', 'thinness 5-9 years',
       'Income composition of resources', 'Schooling'], dtype=object)

Removed the space

In [7]:
pd.get_dummies(life_df2["Status"])

,Developed,Developing
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1
...,...,...
2933,0,1
2934,0,1
2935,0,1
2936,0,1


In [8]:
life_df2= pd.concat([life_df2, pd.get_dummies(life_df["Status"], drop_first=True)], axis=1)
life_df2.head()

,Country,Year,Status,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,...,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling,Developing
0,Afghanistan,2015,Developing,65.0,263.0,62,0.01,71.279624,65.0,1154,...,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1,1
1,Afghanistan,2014,Developing,59.9,271.0,64,0.01,73.523582,62.0,492,...,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0,1
2,Afghanistan,2013,Developing,59.9,268.0,66,0.01,73.219243,64.0,430,...,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9,1
3,Afghanistan,2012,Developing,59.5,272.0,69,0.01,78.184215,67.0,2787,...,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8,1
4,Afghanistan,2011,Developing,59.2,275.0,71,0.01,7.097109,68.0,3013,...,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5,1


### Missing Value

In [9]:
life_df2.isnull().sum()

Country                              0
Year                                 0
Status                               0
Life expectancy                     10
Adult Mortality                     10
infant deaths                        0
Alcohol                            194
percentage expenditure               0
Hepatitis B                        553
Measles                              0
BMI                                 34
under-five deaths                    0
Polio                               19
Total expenditure                  226
Diphtheria                          19
HIV/AIDS                             0
GDP                                448
Population                         652
thinness  1-19 years                34
thinness 5-9 years                  34
Income composition of resources    167
Schooling                          163
Developing                           0
dtype: int64

In [10]:
life_df3=life_df2.copy()

In [11]:
life_df3["Life expectancy"]=life_df3["Life expectancy"].fillna(life_df3.groupby("Country")["Life expectancy"].transform("mean"))
life_df3["Life expectancy"]=life_df3["Life expectancy"].fillna(life_df3["Life expectancy"].mean())

life_df3["Adult Mortality"]=life_df3["Adult Mortality"].fillna(life_df3.groupby("Country")["Adult Mortality"].transform("mean"))
life_df3["Adult Mortality"]=life_df3["Adult Mortality"].fillna(life_df3["Adult Mortality"].mean())

life_df3["Alcohol"]=life_df3["Alcohol"].fillna(life_df3.groupby("Country")["Alcohol"].transform("mean"))
life_df3["Alcohol"]=life_df3["Alcohol"].fillna(life_df3["Alcohol"].mean())

life_df3["Hepatitis B"]=life_df3["Hepatitis B"].fillna(life_df3.groupby("Country")["Hepatitis B"].transform("mean"))
life_df3["Hepatitis B"]=life_df3["Hepatitis B"].fillna(life_df3["Hepatitis B"].mean())

life_df3["BMI"]=life_df3["BMI"].fillna(life_df3.groupby("Country")["BMI"].transform("mean"))
life_df3["BMI"]=life_df3["BMI"].fillna(life_df3["BMI"].mean())

life_df3["Polio"]=life_df3["Polio"].fillna(life_df3.groupby("Country")["Polio"].transform("mean"))
life_df3["Polio"]=life_df3["Polio"].fillna(life_df3["Polio"].mean())

life_df3["Total expenditure"]=life_df3["Total expenditure"].fillna(life_df3.groupby("Country")["Total expenditure"].transform("mean"))
life_df3["Total expenditure"]=life_df3["Total expenditure"].fillna(life_df3["Total expenditure"].mean())

life_df3["Diphtheria"]=life_df3["Diphtheria"].fillna(life_df3.groupby("Country")["Diphtheria"].transform("mean"))
life_df3["Diphtheria"]=life_df3["Diphtheria"].fillna(life_df3["Diphtheria"].mean())

life_df3["GDP"]=life_df3["GDP"].fillna(life_df3.groupby("Country")["GDP"].transform("mean"))
life_df3["GDP"]=life_df3["GDP"].fillna(life_df3["GDP"].mean())

life_df3["Population"]=life_df3["Population"].fillna(life_df3.groupby("Country")["Population"].transform("mean"))
life_df3["Population"]=life_df3["Population"].fillna(life_df3["Population"].mean())

life_df3["thinness  1-19 years"]=life_df3["thinness  1-19 years"].fillna(life_df3.groupby("Country")["thinness  1-19 years"].transform("mean"))
life_df3["thinness  1-19 years"]=life_df3["thinness  1-19 years"].fillna(life_df3["thinness  1-19 years"].mean())

life_df3["thinness 5-9 years"]=life_df3["thinness 5-9 years"].fillna(life_df3.groupby("Country")["thinness 5-9 years"].transform("mean"))
life_df3["thinness 5-9 years"]=life_df3["thinness 5-9 years"].fillna(life_df3["thinness 5-9 years"].mean())

life_df3["Income composition of resources"]=life_df3["Income composition of resources"].fillna(life_df3.groupby("Country")["Income composition of resources"].transform("mean"))
life_df3["Income composition of resources"]=life_df3["Income composition of resources"].fillna(life_df3["Income composition of resources"].mean())

life_df3["Schooling"]=life_df3["Schooling"].fillna(life_df3.groupby("Country")["Schooling"].transform("mean"))
life_df3["Schooling"]=life_df3["Schooling"].fillna(life_df3["Schooling"].mean())

In [12]:
life_df3.isnull().sum()

Country                            0
Year                               0
Status                             0
Life expectancy                    0
Adult Mortality                    0
infant deaths                      0
Alcohol                            0
percentage expenditure             0
Hepatitis B                        0
Measles                            0
BMI                                0
under-five deaths                  0
Polio                              0
Total expenditure                  0
Diphtheria                         0
HIV/AIDS                           0
GDP                                0
Population                         0
thinness  1-19 years               0
thinness 5-9 years                 0
Income composition of resources    0
Schooling                          0
Developing                         0
dtype: int64

## 1. Classification

**Train Model**

In [13]:
y_dt = life_df3['Developing']
X_dt = life_df3.drop(['Developing', 'Year', 'Status', 'Country'], axis=1)

In [14]:
X_dt_train, X_dt_test, y_dt_train, y_dt_test =  train_test_split(X_dt, y_dt, test_size=0.20, random_state=111)

In [15]:
decision_tree = DecisionTreeClassifier(criterion='entropy', max_depth=3,random_state = 100)
decision_tree.fit(X_dt_train, y_dt_train)

DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=100)

**Prediction**

In [16]:
y_pred_dt_train = decision_tree.predict(X_dt_train)
y_pred_dt_test = decision_tree.predict(X_dt_test)


In [17]:
accuracy_dt=accuracy_score(y_pred_dt_train, y_dt_train )
print("Decision Tree Accuracy        : {:.6f}".format(accuracy_dt))

Decision Tree Accuracy        : 0.936170


In [18]:
print("R-squared of the model in training set is: {}".format(decision_tree.score(X_dt_train, y_dt_train)))
print("-----Test set statistics-----")
print("R-squared of the model in test set is: {}".format(decision_tree.score(X_dt_test, y_dt_test)))


R-squared of the model in training set is: 0.9361702127659575
-----Test set statistics-----
R-squared of the model in test set is: 0.935374149659864


**Performance Measuring**

In [21]:
print(classification_report(y_dt_test, y_pred_dt_test))

print("f1_score() value        : {:.2f}".format(f1_score(y_dt_test, y_pred_dt_test)))
print("recall_score() value    : {:.2f}".format(recall_score(y_dt_test, y_pred_dt_test)))
print("precision_score() value : {:.2f}".format(precision_score(y_dt_test, y_pred_dt_test)))
print('\n')

metrics = precision_recall_fscore_support(y_dt_test, y_pred_dt_test)
print("Precision  :" , metrics[0]) 
print("Recall     :" , metrics[1]) 
print("F1 Score   :" , metrics[2]) 

              precision    recall  f1-score   support

           0       0.73      0.96      0.83        96
           1       0.99      0.93      0.96       492

    accuracy                           0.94       588
   macro avg       0.86      0.94      0.89       588
weighted avg       0.95      0.94      0.94       588

f1_score() value        : 0.96
recall_score() value    : 0.93
precision_score() value : 0.99


Precision  : [0.73015873 0.99134199]
Recall     : [0.95833333 0.93089431]
F1 Score   : [0.82882883 0.96016771]


**KFold - Cross Validation**

**Train**

In [23]:
cvscores_2 = cross_val_score(decision_tree, X_dt_train, y_dt_train, cv=2)
print('cv =  2 ---> Average cross validation score : ', np.mean(cvscores_2))

cvscores_3 = cross_val_score(decision_tree, X_dt_train, y_dt_train, cv=3)
print('cv =  3 ---> Average cross validation score : ', np.mean(cvscores_3))

cvscores_4 = cross_val_score(decision_tree, X_dt_train, y_dt_train, cv=4)
print('cv =  4 ---> Average cross validation score : ', np.mean(cvscores_4))

cvscores_5 = cross_val_score(decision_tree, X_dt_train, y_dt_train, cv=5)
print('cv =  5 ---> Average cross validation score : ', np.mean(cvscores_5))

cvscores_6 = cross_val_score(decision_tree, X_dt_train, y_dt_train, cv=6)
print('cv =  6 ---> Average cross validation score : ', np.mean(cvscores_6))

cvscores_7 = cross_val_score(decision_tree, X_dt_train, y_dt_train, cv=15)
print('cv = 7 ---> Average cross validation score  : ', np.mean(cvscores_7))

cv =  2 ---> Average cross validation score :  0.9182978723404256
cv =  3 ---> Average cross validation score :  0.9191498119043274
cv =  4 ---> Average cross validation score :  0.9242538156659598
cv =  5 ---> Average cross validation score :  0.9289361702127661
cv =  6 ---> Average cross validation score :  0.9246742175130923
cv = 7 ---> Average cross validation score  :  0.9251075181011486


**Test**

In [24]:
cvscores_2 = cross_val_score(decision_tree, X_dt_test, y_dt_test, cv=2)
print('cv =  2 ---> Average cross validation score : ', np.mean(cvscores_2))

cvscores_3 = cross_val_score(decision_tree, X_dt_test, y_dt_test, cv=3)
print('cv =  3 ---> Average cross validation score : ', np.mean(cvscores_3))

cvscores_4 = cross_val_score(decision_tree, X_dt_test, y_dt_test, cv=4)
print('cv =  4 ---> Average cross validation score : ', np.mean(cvscores_4))

cvscores_5 = cross_val_score(decision_tree, X_dt_test, y_dt_test, cv=5)
print('cv =  5 ---> Average cross validation score : ', np.mean(cvscores_5))

cvscores_6 = cross_val_score(decision_tree, X_dt_test, y_dt_test, cv=6)
print('cv =  6 ---> Average cross validation score : ', np.mean(cvscores_6))

cvscores_7 = cross_val_score(decision_tree, X_dt_test, y_dt_test, cv=10)
print('cv = 7 ---> Average cross validation score  : ', np.mean(cvscores_7))

cv =  2 ---> Average cross validation score :  0.9251700680272109
cv =  3 ---> Average cross validation score :  0.9217687074829932
cv =  4 ---> Average cross validation score :  0.9166666666666667
cv =  5 ---> Average cross validation score :  0.9132695929306098
cv =  6 ---> Average cross validation score :  0.9047619047619048
cv = 7 ---> Average cross validation score  :  0.9269140853302161


## 2. Regression

**Train Model**

In [25]:
y_dt2 = life_df3['Life expectancy']
X_dt2 = life_df3.drop(['Life expectancy', 'Status', 'Country'], axis=1)

In [26]:
X_dt2_train, X_dt2_test, y_dt2_train, y_dt2_test =  train_test_split(X_dt2, y_dt2, test_size=0.20, random_state=111)

In [27]:
decision_tree2 = DecisionTreeRegressor(max_depth=3,random_state = 100)
decision_tree2.fit(X_dt2_train, y_dt2_train) 

DecisionTreeRegressor(max_depth=3, random_state=100)

**Prediction**

In [28]:
y_pred_dt2_train = decision_tree2.predict(X_dt2_train)
y_pred_dt2_test = decision_tree2.predict(X_dt2_test)


In [29]:
print("R-squared of the model in training set is: {}".format(decision_tree2.score(X_dt2_train, y_dt2_train)))
print("-----Test set statistics-----")
print("R-squared of the model in test set is: {}".format(decision_tree2.score(X_dt2_test, y_dt2_test)))
print("Mean absolute error (MAE) of the prediction is: {}".format(mean_absolute_error(y_dt2_test, y_pred_dt2_test)))
print("Mean squared error (MSE) of the prediction is: {}".format(mse(y_dt2_test, y_pred_dt2_test)))
print("Root mean squared error (RMSE) of the prediction is: {}".format(rmse(y_dt2_test, y_pred_dt2_test)))
print("Mean absolute percentage error (MAPE) of the prediction is: {}".format(np.mean(np.abs((y_dt2_test - y_pred_dt2_test) / y_dt2_test)) * 100))

R-squared of the model in training set is: 0.8240643615821139
-----Test set statistics-----
R-squared of the model in test set is: 0.815861306183393
Mean absolute error (MAE) of the prediction is: 2.963655078653932
Mean squared error (MSE) of the prediction is: 16.030229556412966
Root mean squared error (RMSE) of the prediction is: 4.003776911419137
Mean absolute percentage error (MAPE) of the prediction is: 4.502807815780812


**KFold - Cross Validation**

**Train**

In [30]:
cvscores_2 = cross_val_score(decision_tree2, X_dt2_train, y_dt2_train, cv=2)
print('cv =  2 ---> Average cross validation score : ', np.mean(cvscores_2))

cvscores_3 = cross_val_score(decision_tree2, X_dt2_train, y_dt2_train, cv=3)
print('cv =  3 ---> Average cross validation score : ', np.mean(cvscores_3))

cvscores_4 = cross_val_score(decision_tree2, X_dt2_train, y_dt2_train, cv=4)
print('cv =  4 ---> Average cross validation score : ', np.mean(cvscores_4))

cvscores_5 = cross_val_score(decision_tree2, X_dt2_train, y_dt2_train, cv=5)
print('cv =  5 ---> Average cross validation score : ', np.mean(cvscores_5))

cvscores_6 = cross_val_score(decision_tree2, X_dt2_train, y_dt2_train, cv=6)
print('cv =  6 ---> Average cross validation score : ', np.mean(cvscores_6))

cvscores_7 = cross_val_score(decision_tree2, X_dt2_train, y_dt2_train, cv=15)
print('cv = 7 ---> Average cross validation score  : ', np.mean(cvscores_7))

cv =  2 ---> Average cross validation score :  0.8003124418723042
cv =  3 ---> Average cross validation score :  0.8021054816621414
cv =  4 ---> Average cross validation score :  0.804489833214666
cv =  5 ---> Average cross validation score :  0.8093954345689788
cv =  6 ---> Average cross validation score :  0.8028858141034788
cv = 7 ---> Average cross validation score  :  0.8032463784114938


**Test**

In [31]:
cvscores_2 = cross_val_score(decision_tree2, X_dt2_test, y_dt2_test, cv=2)
print('cv =  2 ---> Average cross validation score : ', np.mean(cvscores_2))

cvscores_3 = cross_val_score(decision_tree2, X_dt2_test, y_dt2_test, cv=3)
print('cv =  3 ---> Average cross validation score : ', np.mean(cvscores_3))

cvscores_4 = cross_val_score(decision_tree2, X_dt2_test, y_dt2_test, cv=4)
print('cv =  4 ---> Average cross validation score : ', np.mean(cvscores_4))

cvscores_5 = cross_val_score(decision_tree2, X_dt2_test, y_dt2_test, cv=5)
print('cv =  5 ---> Average cross validation score : ', np.mean(cvscores_5))

cvscores_6 = cross_val_score(decision_tree2, X_dt2_test, y_dt2_test, cv=6)
print('cv =  6 ---> Average cross validation score : ', np.mean(cvscores_6))

cvscores_7 = cross_val_score(decision_tree2, X_dt2_test, y_dt2_test, cv=15)
print('cv = 7 ---> Average cross validation score  : ', np.mean(cvscores_7))

cv =  2 ---> Average cross validation score :  0.7951718369579226
cv =  3 ---> Average cross validation score :  0.8233237196662699
cv =  4 ---> Average cross validation score :  0.8001321599006173
cv =  5 ---> Average cross validation score :  0.8168510660794143
cv =  6 ---> Average cross validation score :  0.81379051705478
cv = 7 ---> Average cross validation score  :  0.8004044652425905
